In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline
import seaborn as sn
import plotly.express as px

### Suicide Rates - All Groups

In [2]:
overall_suicide =ddf.read_csv("suicide_rates_analysis_7020/overall_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
overall_suicide.head()

,year,month,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,181440097,1869,1.030092,1
1,1970,Feb,181780664,1794,0.986904,2
2,1970,Mar,182121214,1946,1.068519,3
3,1970,Apr,182461775,2096,1.148734,4
4,1970,May,182801836,2097,1.147144,5


In [3]:
firearm_suicide =ddf.read_csv("suicide_rates_analysis_7020/firearm_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
firearm_suicide.head()

,year,month,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,181440097,953,0.525242,1
1,1970,Feb,181780664,910,0.500603,2
2,1970,Mar,182121214,949,0.521082,3
3,1970,Apr,182461775,1048,0.574367,4
4,1970,May,182801836,1034,0.565640,5


In [4]:
nonfirearm_suicide =ddf.read_csv("suicide_rates_analysis_7020/nonfirearm_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
nonfirearm_suicide.head()

,year,month,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,181440097,916,0.504850,1
1,1970,Feb,181780664,884,0.486300,2
2,1970,Mar,182121214,997,0.547438,3
3,1970,Apr,182461775,1048,0.574367,4
4,1970,May,182801836,1063,0.581504,5


#### yearly suicide rates per 100,000 population

In [5]:
pop = overall_suicide[['year', 'pop', 'month_code']]
pop_total = pop[pop['month_code'] == 12]
pop_total.head()

,year,pop,month_code
11,1970,185186282,12
23,1971,188303852,12
35,1972,190864825,12
47,1973,192218457,12
59,1974,194874242,12


In [6]:
pop_total.shape

(51, 3)

In [7]:
overall_death = overall_suicide[['year', 'suicide_death']].groupby('year').sum()
overall_death = overall_death.reset_index()
overall_death.head()

,year,suicide_death
0,1970,23500
1,1971,24122
2,1972,25028
3,1973,25137
4,1974,25708


In [8]:
firearm_death = firearm_suicide[['year', 'suicide_death']].groupby('year').sum()
firearm_death = firearm_death.reset_index()
firearm_death.head()

,year,suicide_death
0,1970,11777
1,1971,12293
2,1972,13352
3,1973,13323
4,1974,14348


In [9]:
nonfirearm_death = nonfirearm_suicide[['year', 'suicide_death']].groupby('year').sum()
nonfirearm_death = nonfirearm_death.reset_index()
nonfirearm_death.head()

,year,suicide_death
0,1970,11723
1,1971,11829
2,1972,11676
3,1973,11814
4,1974,11360


In [10]:
overall_death = overall_death.rename(columns = {'suicide_death': 'overall_death'})
firearm_death = firearm_death.rename(columns = {'suicide_death': 'firearm_death'})
nonfirearm_death = nonfirearm_death.rename(columns = {'suicide_death': 'nonfirearm_death'})

In [11]:
suicide_yearly = pop_total.merge(overall_death, on = 'year', how = 'inner')
suicide_yearly = suicide_yearly.merge(firearm_death, on = 'year', how = 'inner')
suicide_yearly = suicide_yearly.merge(nonfirearm_death, on = 'year', how = 'inner')
suicide_yearly.head()

,year,pop,month_code,overall_death,firearm_death,nonfirearm_death
0,1970,185186282,12,23500,11777,11723
1,1971,188303852,12,24122,12293,11829
2,1972,190864825,12,25028,13352,11676
3,1973,192218457,12,25137,13323,11814
4,1974,194874242,12,25708,14348,11360


In [12]:
suicide_yearly['overall_smr'] = (suicide_yearly['overall_death'] / suicide_yearly['pop']) * 100000
suicide_yearly['firearm_smr'] = (suicide_yearly['firearm_death'] / suicide_yearly['pop']) * 100000
suicide_yearly['nonfirearm_smr'] = (suicide_yearly['nonfirearm_death'] / suicide_yearly['pop']) * 100000
suicide_yearly.head()

,year,pop,month_code,overall_death,firearm_death,nonfirearm_death,overall_smr,firearm_smr,nonfirearm_smr
0,1970,185186282,12,23500,11777,11723,12.689925,6.359542,6.330383
1,1971,188303852,12,24122,12293,11829,12.810147,6.528279,6.281868
2,1972,190864825,12,25028,13352,11676,13.112945,6.995527,6.117418
3,1973,192218457,12,25137,13323,11814,13.077308,6.931176,6.146132
4,1974,194874242,12,25708,14348,11360,13.192097,7.362697,5.829400


In [13]:
suicide_yearly.head()

,year,pop,month_code,overall_death,firearm_death,nonfirearm_death,overall_smr,firearm_smr,nonfirearm_smr
0,1970,185186282,12,23500,11777,11723,12.689925,6.359542,6.330383
1,1971,188303852,12,24122,12293,11829,12.810147,6.528279,6.281868
2,1972,190864825,12,25028,13352,11676,13.112945,6.995527,6.117418
3,1973,192218457,12,25137,13323,11814,13.077308,6.931176,6.146132
4,1974,194874242,12,25708,14348,11360,13.192097,7.362697,5.829400


In [14]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = suicide_yearly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Create a new plot with a title and axis labels
p = figure(title="Yearly Suicide Rates per 100,000 from 1970 to 2020", x_axis_label='year', y_axis_label='Suicide Rates Per 100,000', x_range=(1970, 2020))

# Add a line renderer for each suicide rates
p.line('year', 'firearm_smr', source=source, line_width=2, line_color="red", legend_label="Firearm")
p.line('year', 'nonfirearm_smr', source=source, line_width=2, line_color="blue", legend_label="Non-Firearm")
p.line('year', 'overall_smr', source=source, line_width=4, line_color="green", legend_label="Overall")

# Add HoverTool
hover = HoverTool(
    tooltips=[
        ("year", "@year"),
        ("firearm_smr", "@{firearm_smr}"),
        ("nonfirearm_smr", "@{nonfirearm_smr}"),
        ("overall_smr", "@{overall_smr}")
    ]
)

p.add_tools(hover)
p.legend.location = "top_right"
p.y_range.start = 0
p.y_range.end = 20
# Show the result
show(p)

ModuleNotFoundError: No module named 'bokeh'

In [ ]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = suicide_yearly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['firearm_smr', 'nonfirearm_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=(1970, 2020), plot_height=400, title="Yearly Suicide Rates from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='year', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='year', width=0.9, color= ['orange', 'green'], source=source,
             legend_label= ['Firearm', 'Non-Firearm'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 20
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"

# Show the result
show(p)

### Monthly Suicide Rates from 1970 to 2020 - All Groups

In [15]:
overall_month = overall_suicide[['year', 'month', 'suicide_rate']]
firearm_month = firearm_suicide[['year', 'month', 'suicide_rate']]
nonfirearm_month = nonfirearm_suicide[['year', 'month', 'suicide_rate']]
overall_month = overall_month.rename(columns = {'suicide_rate': 'overall_smr'})
firearm_month = firearm_month.rename(columns = {'suicide_rate': 'firearm_smr'})
nonfirearm_month = nonfirearm_month.rename(columns = {'suicide_rate': 'nonfirearm_smr'})
suicide_monthly = overall_month.merge(firearm_month, on = ['year', 'month'], how = 'inner')
suicide_monthly = suicide_monthly.merge(nonfirearm_month, on = ['year', 'month'], how = 'inner')
suicide_monthly.head()

,year,month,overall_smr,firearm_smr,nonfirearm_smr
0,1970,Jan,1.030092,0.525242,0.504850
1,1970,Feb,0.986904,0.500603,0.486300
2,1970,Mar,1.068519,0.521082,0.547438
3,1970,Apr,1.148734,0.574367,0.574367
4,1970,May,1.147144,0.565640,0.581504


In [16]:
suicide_monthly = suicide_monthly.groupby('month').mean()
suicide_monthly.head()

,year,overall_smr,firearm_smr,nonfirearm_smr
month,,,,
Apr,1995.0,1.150810,0.630303,0.520508
Aug,1995.0,1.175591,0.637623,0.537968
Dec,1995.0,1.052475,0.587780,0.464696
Feb,1995.0,1.031992,0.575394,0.456599
Jan,1995.0,1.128548,0.630133,0.498415


In [17]:
suicide_monthly = suicide_monthly.reset_index()
suicide_monthly = suicide_monthly[['month', 'overall_smr', 'firearm_smr', 'nonfirearm_smr']]
suicide_monthly

,month,overall_smr,firearm_smr,nonfirearm_smr
0,Apr,1.150810,0.630303,0.520508
1,Aug,1.175591,0.637623,0.537968
2,Dec,1.052475,0.587780,0.464696
3,Feb,1.031992,0.575394,0.456599
4,Jan,1.128548,0.630133,0.498415
5,Jul,1.176744,0.643885,0.532858
6,Jun,1.144430,0.620220,0.524209
7,Mar,1.172092,0.643090,0.529003
8,May,1.184873,0.641393,0.543480
9,Nov,1.069227,0.588304,0.480923


In [18]:
suicide_monthly['month_code'] = suicide_monthly['month']
replace_month = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 
                 'Oct': 10, 'Nov': 11, 'Dec': 12}
suicide_monthly['month_code'] = suicide_monthly['month_code'].replace(replace_month)
suicide_monthly = suicide_monthly.sort_values(['month_code'])
suicide_monthly

,month,overall_smr,firearm_smr,nonfirearm_smr,month_code
4,Jan,1.128548,0.630133,0.498415,1
3,Feb,1.031992,0.575394,0.456599,2
7,Mar,1.172092,0.643090,0.529003,3
0,Apr,1.150810,0.630303,0.520508,4
8,May,1.184873,0.641393,0.543480,5
6,Jun,1.144430,0.620220,0.524209,6
5,Jul,1.176744,0.643885,0.532858,7
1,Aug,1.175591,0.637623,0.537968,8
11,Sep,1.124047,0.608247,0.515800,9
10,Oct,1.132013,0.615541,0.516472,10


In [19]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = suicide_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Create a new plot with a title and axis labels
p = figure(x_range=df['month'], plot_height=400, title="Average Monthly Suicide Rates from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='month', y_axis_label='Suicide Rates per 100,000')

# Add a line renderer for each suicide rates
p.line('month', 'firearm_smr', source=source, line_width=2, line_color="red", legend_label="Firearm")
p.line('month', 'nonfirearm_smr', source=source, line_width=2, line_color="blue", legend_label="Non-Firearm")
p.line('month', 'overall_smr', source=source, line_width=2, line_color="green", legend_label="Overall")

# Set some properties
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.y_range.end = 1.6
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend
# Show the result
show(p)

ModuleNotFoundError: No module named 'bokeh'

In [20]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = suicide_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['firearm_smr', 'nonfirearm_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=df['month'], plot_height=400, title="Average Suicide Rates in Each Month from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='month', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='month', width=0.9, color= ['orange', 'green'], source=source,
             legend_label=['Firearm', 'Non-Firearm'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 1.6
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"

# Show the result
show(p)


ModuleNotFoundError: No module named 'bokeh'

#### Suicide rates in Gender Groups

In [21]:
female_overall =ddf.read_csv("suicide_rates_analysis_7020/overall_female_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
female_overall.head()

,year,month,sex,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,Female,92950207,521,0.560515,1
1,1970,Feb,Female,93200437,507,0.543989,2
2,1970,Mar,Female,93450699,588,0.629209,3
3,1970,Apr,Female,93700947,617,0.658478,4
4,1970,May,Female,93950668,623,0.663114,5


In [22]:
female_firearm =ddf.read_csv("suicide_rates_analysis_7020/firearm_female_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
female_firearm.head()

,year,month,sex,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,Female,92950207,160,0.172135,1
1,1970,Feb,Female,93200437,126,0.135192,2
2,1970,Mar,Female,93450699,178,0.190475,3
3,1970,Apr,Female,93700947,186,0.198504,4
4,1970,May,Female,93950668,191,0.203298,5


In [23]:
female_nonfire =ddf.read_csv("suicide_rates_analysis_7020/nonfire_female_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
female_nonfire.head()

,year,month,sex,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,Female,92950207,361,0.388380,1
1,1970,Feb,Female,93200437,381,0.408796,2
2,1970,Mar,Female,93450699,410,0.438734,3
3,1970,Apr,Female,93700947,431,0.459974,4
4,1970,May,Female,93950668,432,0.459816,5


In [24]:
pop_female = female_overall[['year', 'pop', 'month_code']]
pop_female = pop_female[pop_female['month_code'] == 12]
pop_female.head()

,year,pop,month_code
11,1970,95702877,12
23,1971,96891094,12
35,1972,97688996,12
47,1973,99147665,12
59,1974,100144071,12


In [25]:
male_overall =ddf.read_csv("suicide_rates_analysis_7020/overall_male_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
male_overall.head()

,year,month,sex,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,Male,88488426,1348,1.523363,1
1,1970,Feb,Male,88578878,1287,1.452942,2
2,1970,Mar,Male,88669362,1358,1.531532,3
3,1970,Apr,Male,88759799,1479,1.666295,4
4,1970,May,Male,88849743,1474,1.658981,5


In [26]:
male_firearm =ddf.read_csv("suicide_rates_analysis_7020/firearm_male_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
male_firearm.head()

,year,month,sex,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,Male,88488426,793,0.896162,1
1,1970,Feb,Male,88578878,784,0.885087,2
2,1970,Mar,Male,88669362,771,0.869522,3
3,1970,Apr,Male,88759799,862,0.971160,4
4,1970,May,Male,88849743,843,0.948793,5


In [27]:
male_nonfire =ddf.read_csv("suicide_rates_analysis_7020/nonfire_male_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
male_nonfire.head()

,year,month,sex,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,Male,88488426,555,0.627201,1
1,1970,Feb,Male,88578878,503,0.567855,2
2,1970,Mar,Male,88669362,587,0.662010,3
3,1970,Apr,Male,88759799,617,0.695135,4
4,1970,May,Male,88849743,631,0.710188,5


In [28]:
pop_male = male_overall[['year', 'pop', 'month_code']]
pop_male = pop_male[pop_male['month_code'] == 12]
pop_male.head()

,year,pop,month_code
11,1970,89483405,12
23,1971,91412758,12
35,1972,93175829,12
47,1973,93070792,12
59,1974,94730171,12


In [29]:
female_overall = female_overall.rename(columns = {'suicide_death': 'female_overall_death'})
female_firearm = female_firearm.rename(columns = {'suicide_death': 'female_firearm_death'})
female_nonfire = female_nonfire.rename(columns = {'suicide_death': 'female_nonfire_death'})
male_overall = male_overall.rename(columns = {'suicide_death': 'male_overall_death'})
male_firearm = male_firearm.rename(columns = {'suicide_death': 'male_firearm_death'})
male_nonfire = male_nonfire.rename(columns = {'suicide_death': 'male_nonfire_death'})

In [30]:
female_overall_year = female_overall[['year', 'female_overall_death']].groupby(['year']).sum()
female_overall_year = female_overall_year.reset_index()

In [31]:
female_overall_year.head()

,year,female_overall_death
0,1970,6852
1,1971,7238
2,1972,7244
3,1973,7016
4,1974,7098


In [32]:
female_firearm_year = female_firearm[['year', 'female_firearm_death']].groupby(['year']).sum()
female_firearm_year = female_firearm_year.reset_index()
female_nonfire_year = female_nonfire[['year', 'female_nonfire_death']].groupby(['year']).sum()
female_nonfire_year = female_nonfire_year.reset_index()
male_overall_year = male_overall[['year', 'male_overall_death']].groupby(['year']).sum()
male_overall_year = male_overall_year.reset_index()
male_firearm_year = male_firearm[['year', 'male_firearm_death']].groupby(['year']).sum()
male_firearm_year = male_firearm_year.reset_index()
male_nonfire_year = male_nonfire[['year', 'male_nonfire_death']].groupby(['year']).sum()
male_nonfire_year = male_nonfire_year.reset_index()

In [33]:
female_monthly = pop_female.merge(female_overall_year, on = ['year'], how = 'inner')
female_monthly = female_monthly.merge(female_firearm_year, on = ['year'], how = 'inner')
female_monthly = female_monthly.merge(female_nonfire_year, on = ['year'], how = 'inner')
female_monthly.head()

,year,pop,month_code,female_overall_death,female_firearm_death,female_nonfire_death
0,1970,95702877,12,6852,2068,4784
1,1971,96891094,12,7238,2317,4921
2,1972,97688996,12,7244,2498,4746
3,1973,99147665,12,7016,2261,4755
4,1974,100144071,12,7098,2533,4565


In [34]:
female_monthly['female_overall_smr'] = (female_monthly['female_overall_death']/ female_monthly['pop']) * 100000
female_monthly['female_firearm_smr'] = (female_monthly['female_firearm_death']/ female_monthly['pop']) * 100000
female_monthly['female_nonfire_smr'] = (female_monthly['female_nonfire_death']/ female_monthly['pop']) * 100000

In [35]:
male_monthly = pop_male.merge(male_overall_year, on = ['year'], how = 'inner')
male_monthly = male_monthly.merge(male_firearm_year, on = ['year'], how = 'inner')
male_monthly = male_monthly.merge(male_nonfire_year, on = ['year'], how = 'inner')
male_monthly['male_overall_smr'] = (male_monthly['male_overall_death']/ male_monthly['pop']) * 100000
male_monthly['male_firearm_smr'] = (male_monthly['male_firearm_death']/ male_monthly['pop']) * 100000
male_monthly['male_nonfire_smr'] = (male_monthly['male_nonfire_death']/ male_monthly['pop']) * 100000
male_monthly.head()

,year,pop,month_code,male_overall_death,male_firearm_death,male_nonfire_death,male_overall_smr,male_firearm_smr,male_nonfire_smr
0,1970,89483405,12,16648,9709,6939,18.604567,10.850056,7.754510
1,1971,91412758,12,16884,9976,6908,18.470070,10.913138,7.556932
2,1972,93175829,12,17784,10854,6930,19.086495,11.648944,7.437551
3,1973,93070792,12,18121,11062,7059,19.470125,11.885576,7.584549
4,1974,94730171,12,18610,11815,6795,19.645272,12.472267,7.173005


In [36]:
suicide_yearly.head()

,year,pop,month_code,overall_death,firearm_death,nonfirearm_death,overall_smr,firearm_smr,nonfirearm_smr
0,1970,185186282,12,23500,11777,11723,12.689925,6.359542,6.330383
1,1971,188303852,12,24122,12293,11829,12.810147,6.528279,6.281868
2,1972,190864825,12,25028,13352,11676,13.112945,6.995527,6.117418
3,1973,192218457,12,25137,13323,11814,13.077308,6.931176,6.146132
4,1974,194874242,12,25708,14348,11360,13.192097,7.362697,5.829400


In [37]:
gender_suicide = female_monthly[['year', 'female_overall_smr', 'female_firearm_smr', 'female_nonfire_smr']].merge(male_monthly[['year', 'male_overall_smr', 'male_firearm_smr', 'male_nonfire_smr']], on = 'year', how = 'inner')
gender_suicide = gender_suicide.merge(suicide_yearly[['year', 'overall_smr', 'firearm_smr', 'nonfirearm_smr']], on = 'year', how = 'inner')
gender_suicide.head()

,year,female_overall_smr,female_firearm_smr,female_nonfire_smr,male_overall_smr,male_firearm_smr,male_nonfire_smr,overall_smr,firearm_smr,nonfirearm_smr
0,1970,7.159659,2.160855,4.998805,18.604567,10.850056,7.754510,12.689925,6.359542,6.330383
1,1971,7.470243,2.391345,5.078898,18.470070,10.913138,7.556932,12.810147,6.528279,6.281868
2,1972,7.415369,2.557095,4.858275,19.086495,11.648944,7.437551,13.112945,6.995527,6.117418
3,1973,7.076314,2.280437,4.795877,19.470125,11.885576,7.584549,13.077308,6.931176,6.146132
4,1974,7.087789,2.529356,4.558433,19.645272,12.472267,7.173005,13.192097,7.362697,5.829400


In [38]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = gender_suicide

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Create a new plot with a title and axis labels
p = figure(title="Yearly Overall Suicide Rates by sex per 100,000 from 1970 to 2020", x_axis_label='year', y_axis_label='Suicide Rates Per 100,000', x_range=(1970, 2020))

# Add a line renderer for each suicide rates
p.line('year', 'female_overall_smr', source=source, line_width=2, line_color="green", legend_label="female")
p.line('year', 'male_overall_smr', source=source, line_width=2, line_color="red", legend_label="male")
p.line('year', 'overall_smr', source=source, line_width=4, line_color="black", legend_label="total average")

# Add HoverTool
hover = HoverTool(
    tooltips=[
        ("year", "@year"),
        ("female_overall_smr", "@{female_overall_smr}"),
        ("male_overall_smr", "@{male_overall_smr}"),
        ("overall_smr", "@{overall_smr}")
    ]
)

p.add_tools(hover)
p.y_range.start = 0
p.y_range.end = 30
p.legend.location = "top_right"
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend
p.legend.background_fill_alpha = 0.5
# Show the result
show(p)

ModuleNotFoundError: No module named 'bokeh'

In [39]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = gender_suicide

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Create a new plot with a title and axis labels
p = figure(title="Yearly Firearm Suicide Rates by sex per 100,000 from 1970 to 2020", x_axis_label='year', y_axis_label='Suicide Rates Per 100,000', x_range=(1970, 2020))

# Add a line renderer for each suicide rates
p.line('year', 'female_firearm_smr', source=source, line_width=2, line_color="green", legend_label="female")
p.line('year', 'male_firearm_smr', source=source, line_width=2, line_color="red", legend_label="male")
p.line('year', 'firearm_smr', source=source, line_width=4, line_color="black", legend_label="total average")

# Add HoverTool
hover = HoverTool(
    tooltips=[
        ("year", "@year"),
        ("female_firearm_smr", "@{female_firearm_smr}"),
        ("male_firearm_smr", "@{male_firearm_smr}"),
        ("firearm_smr", "@{firearm_smr}")
    ]
)


p.add_tools(hover)
p.y_range.start = 0
p.y_range.end = 30
p.legend.location = "top_right"
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend
p.legend.background_fill_alpha = 0.5
# Show the result
show(p)

ModuleNotFoundError: No module named 'bokeh'

In [40]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = gender_suicide

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Create a new plot with a title and axis labels
p = figure(title="Yearly Non-Firearm Suicide Rates by sex per 100,000 from 1970 to 2020", x_axis_label='year', y_axis_label='Suicide Rates Per 100,000', x_range=(1970, 2020))

# Add a line renderer for each suicide rates
p.line('year', 'female_nonfire_smr', source=source, line_width=2, line_color="green", legend_label="female")
p.line('year', 'male_nonfire_smr', source=source, line_width=2, line_color="red", legend_label="male")
p.line('year', 'nonfirearm_smr', source=source, line_width=4, line_color="black", legend_label="total average")

# Add HoverTool
hover = HoverTool(
    tooltips=[
        ("year", "@year"),
        ("female_nonfire_smr", "@{female_nonfire_smr}"),
        ("male_nonfire_smr", "@{male_nonfire_smr}"),
        ("nonfirearm_smr", "@{nonfirearm_smr}")
    ]
)

p.add_tools(hover)
p.y_range.start = 0
p.y_range.end = 30
p.legend.location = "top_right"
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend
p.legend.background_fill_alpha = 0.5
# Show the result
show(p)

ModuleNotFoundError: No module named 'bokeh'

In [41]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = gender_suicide

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['male_overall_smr', 'female_overall_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=(1970, 2020), plot_height=400, title="Yearly Overall Suicide Rates by Sex from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='year', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='year', width=0.9, color= ['orange', 'green'], source=source,
             legend_label= ['male', 'female'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 45
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"

# Show the result
show(p)

ModuleNotFoundError: No module named 'bokeh'

In [42]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = gender_suicide

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['male_firearm_smr', 'female_firearm_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=(1970, 2020), plot_height=400, title="Yearly Firearm Suicide Rates by Sex from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='year', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='year', width=0.9, color= ['orange', 'green'], source=source,
             legend_label= ['male', 'female'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 45
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"

# Show the result
show(p)

ModuleNotFoundError: No module named 'bokeh'

In [43]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = gender_suicide

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['male_nonfire_smr', 'female_nonfire_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=(1970, 2020), plot_height=400, title="Yearly Non-Firearm Suicide Rates by Sex from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='year', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='year', width=0.9, color= ['orange', 'green'], source=source,
             legend_label= ['male', 'female'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 45
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"

# Show the result
show(p)

ModuleNotFoundError: No module named 'bokeh'

In [44]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = gender_suicide

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['female_firearm_smr', 'female_nonfire_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=(1970, 2020), plot_height=400, title="Yearly Average Female Suicide Rates from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='year', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='year', width=0.9, color= ['orange', 'green'], source=source,
             legend_label= ['firearm', 'non-firearm'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 35
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"

# Show the result
show(p)

ModuleNotFoundError: No module named 'bokeh'

In [45]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = gender_suicide

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['male_firearm_smr', 'male_nonfire_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=(1970, 2020), plot_height=400, title="Yearly Average Male Suicide Rates from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='year', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='year', width=0.9, color= ['orange', 'green'], source=source,
             legend_label= ['firearm', 'non-firearm'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 35
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"

# Show the result
show(p)

ModuleNotFoundError: No module named 'bokeh'

In [46]:
suicide_monthly.head()

,month,overall_smr,firearm_smr,nonfirearm_smr,month_code
4,Jan,1.128548,0.630133,0.498415,1
3,Feb,1.031992,0.575394,0.456599,2
7,Mar,1.172092,0.643090,0.529003,3
0,Apr,1.150810,0.630303,0.520508,4
8,May,1.184873,0.641393,0.543480,5


In [47]:
def findMonthlyRate(df, group):
    df_month = df[['year', 'month', 'suicide_rate', 'month_code']].groupby(['month', 'month_code']).mean()
    df_month = df_month.reset_index()
    df_month = df_month[['month', 'month_code', 'suicide_rate']]
    new_name = group + '_smr'
    df_month = df_month.rename(columns = {'suicide_rate': new_name})
    df_month = df_month.sort_values(['month_code'])
    return df_month

In [48]:
female_overall_month = female_overall[['year', 'month', 'suicide_rate', 'month_code']].groupby(['month', 'month_code']).mean()
female_overall_month = female_overall_month.reset_index()
female_overall_month = female_overall_month[['month', 'month_code', 'suicide_rate']]
female_overall_month = female_overall_month.rename(columns = {'suicide_rate': 'female_overall_smr'})
female_overall_month = female_overall_month.sort_values(['month_code'])
female_overall_month

,month,month_code,female_overall_smr
4,Jan,1,0.486593
3,Feb,2,0.447151
7,Mar,3,0.513562
0,Apr,4,0.506102
8,May,5,0.526765
6,Jun,6,0.502109
5,Jul,7,0.509878
1,Aug,8,0.516288
11,Sep,9,0.497895
10,Oct,10,0.505813


In [49]:
male_overall_month = findMonthlyRate(male_overall, 'male_overall')
male_overall_month

,month,month_code,male_overall_smr
4,Jan,1,1.799021
3,Feb,2,1.642809
7,Mar,3,1.859756
0,Apr,4,1.824044
8,May,5,1.871953
6,Jun,6,1.814965
5,Jul,7,1.872866
1,Aug,8,1.863822
11,Sep,9,1.777551
10,Oct,10,1.785696


In [51]:
female_firearm_month = findMonthlyRate(female_firearm, 'female_firearm')
female_firearm_month

,month,month_code,female_firearm_smr
4,Jan,1,0.173506
3,Feb,2,0.159927
7,Mar,3,0.180214
0,Apr,4,0.175222
8,May,5,0.184645
6,Jun,6,0.176334
5,Jul,7,0.179365
1,Aug,8,0.179015
11,Sep,9,0.174643
10,Oct,10,0.179873


In [52]:
male_firearm_month = findMonthlyRate(male_firearm, 'male_firearm')
male_firearm_month

,month,month_code,male_firearm_smr
4,Jan,1,1.107298
3,Feb,2,1.009563
7,Mar,3,1.126751
0,Apr,4,1.105793
8,May,5,1.118537
6,Jun,6,1.083903
5,Jul,7,1.129095
1,Aug,8,1.116665
11,Sep,9,1.061119
10,Oct,10,1.070574


In [53]:
female_nonfirearm_month = findMonthlyRate(female_nonfire, 'female_nonfirearm')
female_nonfirearm_month

,month,month_code,female_nonfirearm_smr
4,Jan,1,0.313087
3,Feb,2,0.287224
7,Mar,3,0.333349
0,Apr,4,0.330880
8,May,5,0.342120
6,Jun,6,0.325775
5,Jul,7,0.330513
1,Aug,8,0.337274
11,Sep,9,0.323251
10,Oct,10,0.325940


In [54]:
male_nonfirearm_month = findMonthlyRate(male_nonfire, 'male_nonfirearm')
male_nonfirearm_month

,month,month_code,male_nonfirearm_smr
4,Jan,1,0.691723
3,Feb,2,0.633246
7,Mar,3,0.733005
0,Apr,4,0.718252
8,May,5,0.753417
6,Jun,6,0.731062
5,Jul,7,0.743771
1,Aug,8,0.747157
11,Sep,9,0.716432
10,Oct,10,0.715121


In [55]:
female_monthly = female_overall_month.merge(female_firearm_month, on = ['month', 'month_code'], how = 'inner')
female_monthly = female_monthly.merge(female_nonfirearm_month, on = ['month', 'month_code'], how = 'inner')
male_monthly = male_overall_month.merge(male_firearm_month, on = ['month', 'month_code'], how = 'inner')
male_monthly = male_monthly.merge(male_nonfirearm_month, on = ['month', 'month_code'], how = 'inner')
gender_monthly = suicide_monthly.merge(female_monthly, on = ['month', 'month_code'], how = 'inner')
gender_monthly = gender_monthly.merge(male_monthly, on = ['month', 'month_code'], how = 'inner')
gender_monthly

,month,overall_smr,firearm_smr,nonfirearm_smr,month_code,female_overall_smr,female_firearm_smr,female_nonfirearm_smr,male_overall_smr,male_firearm_smr,male_nonfirearm_smr
0,Jan,1.128548,0.630133,0.498415,1,0.486593,0.173506,0.313087,1.799021,1.107298,0.691723
1,Feb,1.031992,0.575394,0.456599,2,0.447151,0.159927,0.287224,1.642809,1.009563,0.633246
2,Mar,1.172092,0.643090,0.529003,3,0.513562,0.180214,0.333349,1.859756,1.126751,0.733005
3,Apr,1.150810,0.630303,0.520508,4,0.506102,0.175222,0.330880,1.824044,1.105793,0.718252
4,May,1.184873,0.641393,0.543480,5,0.526765,0.184645,0.342120,1.871953,1.118537,0.753417
5,Jun,1.144430,0.620220,0.524209,6,0.502109,0.176334,0.325775,1.814965,1.083903,0.731062
6,Jul,1.176744,0.643885,0.532858,7,0.509878,0.179365,0.330513,1.872866,1.129095,0.743771
7,Aug,1.175591,0.637623,0.537968,8,0.516288,0.179015,0.337274,1.863822,1.116665,0.747157
8,Sep,1.124047,0.608247,0.515800,9,0.497895,0.174643,0.323251,1.777551,1.061119,0.716432
9,Oct,1.132013,0.615541,0.516472,10,0.505813,0.179873,0.325940,1.785696,1.070574,0.715121


In [56]:
gender_monthly

,month,overall_smr,firearm_smr,nonfirearm_smr,month_code,female_overall_smr,female_firearm_smr,female_nonfirearm_smr,male_overall_smr,male_firearm_smr,male_nonfirearm_smr
0,Jan,1.128548,0.630133,0.498415,1,0.486593,0.173506,0.313087,1.799021,1.107298,0.691723
1,Feb,1.031992,0.575394,0.456599,2,0.447151,0.159927,0.287224,1.642809,1.009563,0.633246
2,Mar,1.172092,0.643090,0.529003,3,0.513562,0.180214,0.333349,1.859756,1.126751,0.733005
3,Apr,1.150810,0.630303,0.520508,4,0.506102,0.175222,0.330880,1.824044,1.105793,0.718252
4,May,1.184873,0.641393,0.543480,5,0.526765,0.184645,0.342120,1.871953,1.118537,0.753417
5,Jun,1.144430,0.620220,0.524209,6,0.502109,0.176334,0.325775,1.814965,1.083903,0.731062
6,Jul,1.176744,0.643885,0.532858,7,0.509878,0.179365,0.330513,1.872866,1.129095,0.743771
7,Aug,1.175591,0.637623,0.537968,8,0.516288,0.179015,0.337274,1.863822,1.116665,0.747157
8,Sep,1.124047,0.608247,0.515800,9,0.497895,0.174643,0.323251,1.777551,1.061119,0.716432
9,Oct,1.132013,0.615541,0.516472,10,0.505813,0.179873,0.325940,1.785696,1.070574,0.715121


In [57]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = gender_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Create a new plot with a title and axis labels
p = figure(x_range=df['month'], plot_height=400, title="Average Monthly Overall Suicide Rates from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='month', y_axis_label='Suicide Rates per 100,000')

# Add a line renderer for each suicide rates
p.line('month', 'female_overall_smr', source=source, line_width=2, line_color="blue", legend_label="female")
p.line('month', 'male_overall_smr', source=source, line_width=2, line_color="red", legend_label="male")
p.line('month', 'overall_smr', source=source, line_width=4, line_color="black", legend_label="total average")

# Set some properties
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.y_range.end = 2.5
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend
# Show the result
show(p)

Loading BokehJS ...

In [58]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = gender_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Create a new plot with a title and axis labels
p = figure(x_range=df['month'], plot_height=400, title="Average Monthly Firearm Suicide Rates from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='month', y_axis_label='Suicide Rates per 100,000')

# Add a line renderer for each suicide rates

p.line('month', 'female_firearm_smr', source=source, line_width=2, line_color="blue", legend_label="female")
p.line('month', 'male_firearm_smr', source=source, line_width=2, line_color="red", legend_label="male")
p.line('month', 'firearm_smr', source=source, line_width=4, line_color="black", legend_label="total average")

# Set some properties
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.y_range.end = 2.5
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend

# Show the result
show(p)

Loading BokehJS ...

In [59]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = gender_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Create a new plot with a title and axis labels
p = figure(x_range=df['month'], plot_height=400, title="Average Monthly Non-Firearm Suicide Rates from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='month', y_axis_label='Suicide Rates per 100,000')

# Add a line renderer for each suicide rates

p.line('month', 'female_nonfirearm_smr', source=source, line_width=2, line_color="blue", legend_label="female")
p.line('month', 'male_nonfirearm_smr', source=source, line_width=2, line_color="red", legend_label="male")
p.line('month', 'nonfirearm_smr', source=source, line_width=4, line_color="black", legend_label="total average")

# Set some properties
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.y_range.end = 2.5
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend

# Show the result
show(p)

Loading BokehJS ...

In [60]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = gender_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['male_overall_smr', 'female_overall_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=df['month'], plot_height=400, title="Average Overall Suicide Rates in Each Month from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='month', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='month', width=0.9, color= ['orange', 'green'], source=source,
             legend_label=['male', 'female'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 3.5
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"

# Show the result
show(p)

Loading BokehJS ...

In [61]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = gender_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['male_firearm_smr', 'female_firearm_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=df['month'], plot_height=400, title="Average Firearm Suicide Rates in Each Month from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='month', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='month', width=0.9, color= ['orange', 'green'], source=source,
             legend_label=['male', 'female'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 3.5
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"

# Show the result
show(p)

Loading BokehJS ...

In [62]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = gender_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['male_nonfirearm_smr', 'female_nonfirearm_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=df['month'], plot_height=400, title="Average Non-Firearm Suicide Rates in Each Month from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='month', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='month', width=0.9, color= ['orange', 'green'], source=source,
             legend_label=['male', 'female'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 3.5
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"

# Show the result
show(p)

Loading BokehJS ...

In [63]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = gender_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['female_firearm_smr', 'female_nonfirearm_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=df['month'], plot_height=400, title="Average Female Suicide Rates in Each Month from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='month', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='month', width=0.9, color= ['orange', 'green'], source=source,
             legend_label=['firearm', 'non-firarm'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 2.5
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"

# Show the result
show(p)

Loading BokehJS ...

In [64]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = gender_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['male_firearm_smr', 'male_nonfirearm_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=df['month'], plot_height=400, title="Average Male Suicide Rates in Each Month from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='month', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='month', width=0.9, color= ['orange', 'green'], source=source,
             legend_label=['firearm', 'non-firearm'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 2.5
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"

# Show the result
show(p)

Loading BokehJS ...

### Age Group

In [65]:
age_0024_overall =ddf.read_csv("suicide_rates_analysis_7020/overall_age_0024_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})

age_0024_overall

,year,month,age_range_x,pop,age_range_y,suicide_death,suicide_rate,month_code
0,1970,Jan,age 0-24,83033802,age 0-24,259,0.311921,1
1,1970,Feb,age 0-24,83209526,age 0-24,265,0.318473,2
2,1970,Mar,age 0-24,83385288,age 0-24,254,0.304610,3
3,1970,Apr,age 0-24,83561028,age 0-24,285,0.341068,4
4,1970,May,age 0-24,83736276,age 0-24,272,0.324829,5
...,...,...,...,...,...,...,...,...
607,2020,Aug,age 0-24,93366179,age 0-24,584,0.625494,8
608,2020,Sep,age 0-24,93345798,age 0-24,634,0.679195,9
609,2020,Oct,age 0-24,93325417,age 0-24,638,0.683629,10
610,2020,Nov,age 0-24,93305025,age 0-24,562,0.602326,11


In [66]:
age_0024_overall =ddf.read_csv("suicide_rates_analysis_7020/overall_age_0024_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
age_0024_overall = age_0024_overall.drop('age_range_y', axis = 1)
age_0024_overall = age_0024_overall.rename(columns = {'age_range_x': 'age_range'})
age_0024_overall.head()

,year,month,age_range,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,age 0-24,83033802,259,0.311921,1
1,1970,Feb,age 0-24,83209526,265,0.318473,2
2,1970,Mar,age 0-24,83385288,254,0.304610,3
3,1970,Apr,age 0-24,83561028,285,0.341068,4
4,1970,May,age 0-24,83736276,272,0.324829,5


In [67]:
age_2564_overall =ddf.read_csv("suicide_rates_analysis_7020/overall_age_2564_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
age_2564_overall.head()

,year,month,age_range,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,age 25-64,80572710,1305,1.619655,1
1,1970,Feb,age 25-64,80699970,1232,1.526642,2
2,1970,Mar,age 25-64,80827204,1335,1.651672,3
3,1970,Apr,age 25-64,80954474,1423,1.757778,4
4,1970,May,age 25-64,81081186,1428,1.761198,5


In [68]:
age_65_overall =ddf.read_csv("suicide_rates_analysis_7020/overall_age_65_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
age_65_overall.head()

,year,month,age_range,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,65 and older,17830773,305,1.710526,1
1,1970,Feb,65 and older,17868599,296,1.656537,2
2,1970,Mar,65 and older,17906383,355,1.982533,3
3,1970,Apr,65 and older,17944222,386,2.151110,4
4,1970,May,65 and older,17981511,395,2.196701,5


In [69]:
age_0024_firearm =ddf.read_csv("suicide_rates_analysis_7020/firearm_age_0024_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
age_0024_firearm.head()

,year,month,age_range,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,age 0-24,83033802,117,0.140906,1
1,1970,Feb,age 0-24,83209526,122,0.146618,2
2,1970,Mar,age 0-24,83385288,122,0.146309,3
3,1970,Apr,age 0-24,83561028,136,0.162755,4
4,1970,May,age 0-24,83736276,121,0.144501,5


In [70]:
age_0024_nonfire =ddf.read_csv("suicide_rates_analysis_7020/nonfire_age_0024_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
age_0024_nonfire.head()

,year,month,age_range,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,age 0-24,83033802,142,0.171015,1
1,1970,Feb,age 0-24,83209526,143,0.171855,2
2,1970,Mar,age 0-24,83385288,132,0.158301,3
3,1970,Apr,age 0-24,83561028,149,0.178313,4
4,1970,May,age 0-24,83736276,151,0.180328,5


In [71]:
age_2564_firearm =ddf.read_csv("suicide_rates_analysis_7020/firearm_age_2564_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
age_2564_firearm.head()

,year,month,age_range,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,age 25-64,80572710,670,0.831547,1
1,1970,Feb,age 25-64,80699970,628,0.778191,2
2,1970,Mar,age 25-64,80827204,637,0.788101,3
3,1970,Apr,age 25-64,80954474,717,0.885683,4
4,1970,May,age 25-64,81081186,713,0.879366,5


In [72]:
age_2564_nonfire =ddf.read_csv("suicide_rates_analysis_7020/nonfire_age_2564_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
age_2564_nonfire.head()

,year,month,age_range,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,age 25-64,80572710,635,0.788108,1
1,1970,Feb,age 25-64,80699970,604,0.748451,2
2,1970,Mar,age 25-64,80827204,698,0.863571,3
3,1970,Apr,age 25-64,80954474,706,0.872095,4
4,1970,May,age 25-64,81081186,715,0.881832,5


In [73]:
age_65_nonfire =ddf.read_csv("suicide_rates_analysis_7020/nonfire_age_65_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
age_65_nonfire.head()

,year,month,age_range,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,65 and older,17830773,139,0.779551,1
1,1970,Feb,65 and older,17868599,137,0.766708,2
2,1970,Mar,65 and older,17906383,166,0.927044,3
3,1970,Apr,65 and older,17944222,192,1.069982,4
4,1970,May,65 and older,17981511,195,1.084447,5


In [74]:
age_65_firearm =ddf.read_csv("suicide_rates_analysis_7020/firearm_age_65_monthly_suicide_rate_1970-2020.csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
age_65_firearm.head()

,year,month,age_range,pop,suicide_death,suicide_rate,month_code
0,1970,Jan,65 and older,17830773,166,0.930975,1
1,1970,Feb,65 and older,17868599,159,0.889829,2
2,1970,Mar,65 and older,17906383,189,1.055490,3
3,1970,Apr,65 and older,17944222,194,1.081128,4
4,1970,May,65 and older,17981511,200,1.112254,5


In [75]:
age_0024_pop = age_0024_overall[['year', 'month', 'pop', 'age_range', 'month_code']]
age_0024_pop_yearly = age_0024_pop[age_0024_pop['month_code'] == 12]
age_0024_pop_yearly.head()

,year,month,pop,age_range,month_code
11,1970,Dec,84967089,age 0-24,12
23,1971,Dec,86728328,age 0-24,12
35,1972,Dec,86574634,age 0-24,12
47,1973,Dec,86581713,age 0-24,12
59,1974,Dec,86100981,age 0-24,12


In [76]:
age_2564_pop = age_2564_overall[['year', 'month', 'pop', 'age_range', 'month_code']]
age_2564_pop_yearly = age_2564_pop[age_2564_pop['month_code'] == 12]
age_2564_pop_yearly.head()

,year,month,pop,age_range,month_code
11,1970,Dec,81972436,age 25-64,12
23,1971,Dec,82842024,age 25-64,12
35,1972,Dec,85227680,age 25-64,12
47,1973,Dec,86081930,age 25-64,12
59,1974,Dec,88620156,age 25-64,12


In [77]:
age_65_pop = age_65_overall[['year', 'month', 'pop', 'age_range', 'month_code']]
age_65_pop_yearly = age_65_pop[age_65_pop['month_code'] == 12]
age_65_pop_yearly.head()

,year,month,pop,age_range,month_code
11,1970,Dec,18246757,65 and older,12
23,1971,Dec,18733500,65 and older,12
35,1972,Dec,19062511,65 and older,12
47,1973,Dec,19554814,65 and older,12
59,1974,Dec,20153105,65 and older,12


In [78]:
age_0024_overall = age_0024_overall.rename(columns = {'suicide_death': 'age_0024_overall_death'})
age_0024_firearm = age_0024_firearm.rename(columns = {'suicide_death': 'age_0024_firearm_death'})
age_0024_nonfire = age_0024_nonfire.rename(columns = {'suicide_death': 'age_0024_nonfire_death'})
age_2564_overall = age_2564_overall.rename(columns = {'suicide_death': 'age_2564_overall_death'})
age_2564_firearm = age_2564_firearm.rename(columns = {'suicide_death': 'age_2564_firearm_death'})
age_2564_nonfire = age_2564_nonfire.rename(columns = {'suicide_death': 'age_2564_nonfire_death'})
age_65_overall = age_65_overall.rename(columns = {'suicide_death': 'age_65_overall_death'})
age_65_firearm = age_65_firearm.rename(columns = {'suicide_death': 'age_65_firearm_death'})
age_65_nonfire = age_65_nonfire.rename(columns = {'suicide_death': 'age_65_nonfire_death'})

In [79]:
age_0024_overall_year = age_0024_overall[['year', 'age_0024_overall_death']].groupby(['year']).sum()
age_0024_overall_year = age_0024_overall_year.reset_index()
age_0024_firearm_year = age_0024_firearm[['year', 'age_0024_firearm_death']].groupby(['year']).sum()
age_0024_firearm_year = age_0024_firearm_year.reset_index()
age_0024_nonfire_year = age_0024_nonfire[['year', 'age_0024_nonfire_death']].groupby(['year']).sum()
age_0024_nonfire_year = age_0024_nonfire_year.reset_index()

age_2564_overall_year = age_2564_overall[['year', 'age_2564_overall_death']].groupby(['year']).sum()
age_2564_overall_year = age_2564_overall_year.reset_index()
age_2564_firearm_year = age_2564_firearm[['year', 'age_2564_firearm_death']].groupby(['year']).sum()
age_2564_firearm_year = age_2564_firearm_year.reset_index()
age_2564_nonfire_year = age_2564_nonfire[['year', 'age_2564_nonfire_death']].groupby(['year']).sum()
age_2564_nonfire_year = age_2564_nonfire_year.reset_index()

age_65_overall_year = age_65_overall[['year', 'age_65_overall_death']].groupby(['year']).sum()
age_65_overall_year = age_65_overall_year.reset_index()
age_65_firearm_year = age_65_firearm[['year', 'age_65_firearm_death']].groupby(['year']).sum()
age_65_firearm_year = age_65_firearm_year.reset_index()
age_65_nonfire_year = age_65_nonfire[['year', 'age_65_nonfire_death']].groupby(['year']).sum()
age_65_nonfire_year = age_65_nonfire_year.reset_index()

In [80]:
age_0024_pop.head()

,year,month,pop,age_range,month_code
0,1970,Jan,83033802,age 0-24,1
1,1970,Feb,83209526,age 0-24,2
2,1970,Mar,83385288,age 0-24,3
3,1970,Apr,83561028,age 0-24,4
4,1970,May,83736276,age 0-24,5


In [81]:
age_0024_overall_year.head()

,year,age_0024_overall_death
0,1970,3267
1,1971,3628
2,1972,3988
3,1973,4257
4,1974,4480


In [82]:
age_0024_monthly = age_0024_pop_yearly.merge(age_0024_overall_year, on = ['year'], how = 'inner')
age_0024_monthly = age_0024_monthly.merge(age_0024_firearm_year, on = ['year'], how = 'inner')
age_0024_monthly = age_0024_monthly.merge(age_0024_nonfire_year, on = ['year'], how = 'inner')
age_0024_monthly.head()

,year,month,pop,age_range,month_code,age_0024_overall_death,age_0024_firearm_death,age_0024_nonfire_death
0,1970,Dec,84967089,age 0-24,12,3267,1531,1736
1,1971,Dec,86728328,age 0-24,12,3628,1819,1809
2,1972,Dec,86574634,age 0-24,12,3988,2212,1776
3,1973,Dec,86581713,age 0-24,12,4257,2331,1926
4,1974,Dec,86100981,age 0-24,12,4480,2633,1847


In [83]:
age_2564_monthly = age_2564_pop_yearly.merge(age_2564_overall_year, on = ['year'], how = 'inner')
age_2564_monthly = age_2564_monthly.merge(age_2564_firearm_year, on = ['year'], how = 'inner')
age_2564_monthly = age_2564_monthly.merge(age_2564_nonfire_year, on = ['year'], how = 'inner')
age_2564_monthly.head()

,year,month,pop,age_range,month_code,age_2564_overall_death,age_2564_firearm_death,age_2564_nonfire_death
0,1970,Dec,81972436,age 25-64,12,16047,8082,7965
1,1971,Dec,82842024,age 25-64,12,16186,8234,7952
2,1972,Dec,85227680,age 25-64,12,16650,8828,7822
3,1973,Dec,86081930,age 25-64,12,16548,8674,7874
4,1974,Dec,88620156,age 25-64,12,17000,9349,7651


In [84]:
age_65_monthly = age_65_pop_yearly.merge(age_65_overall_year, on = ['year'], how = 'inner')
age_65_monthly = age_65_monthly.merge(age_65_firearm_year, on = ['year'], how = 'inner')
age_65_monthly = age_65_monthly.merge(age_65_nonfire_year, on = ['year'], how = 'inner')
age_65_monthly.head()

,year,month,pop,age_range,month_code,age_65_overall_death,age_65_firearm_death,age_65_nonfire_death
0,1970,Dec,18246757,65 and older,12,4172,2157,2015
1,1971,Dec,18733500,65 and older,12,4301,2239,2062
2,1972,Dec,19062511,65 and older,12,4384,2306,2078
3,1973,Dec,19554814,65 and older,12,4326,2314,2012
4,1974,Dec,20153105,65 and older,12,4222,2363,1859


In [85]:
age_0024_monthly['age_0024_overall_smr'] = (age_0024_monthly['age_0024_overall_death']/ age_0024_monthly['pop']) * 100000
age_0024_monthly['age_0024_firearm_smr'] = (age_0024_monthly['age_0024_firearm_death']/ age_0024_monthly['pop']) * 100000
age_0024_monthly['age_0024_nonfire_smr'] = (age_0024_monthly['age_0024_nonfire_death']/ age_0024_monthly['pop']) * 100000

age_2564_monthly['age_2564_overall_smr'] = (age_2564_monthly['age_2564_overall_death']/ age_2564_monthly['pop']) * 100000
age_2564_monthly['age_2564_firearm_smr'] = (age_2564_monthly['age_2564_firearm_death']/ age_2564_monthly['pop']) * 100000
age_2564_monthly['age_2564_nonfire_smr'] = (age_2564_monthly['age_2564_nonfire_death']/ age_2564_monthly['pop']) * 100000

age_65_monthly['age_65_overall_smr'] = (age_65_monthly['age_65_overall_death']/ age_65_monthly['pop']) * 100000
age_65_monthly['age_65_firearm_smr'] = (age_65_monthly['age_65_firearm_death']/ age_65_monthly['pop']) * 100000
age_65_monthly['age_65_nonfire_smr'] = (age_65_monthly['age_65_nonfire_death']/ age_65_monthly['pop']) * 100000

In [86]:
age_0024_monthly.head()

,year,month,pop,age_range,month_code,age_0024_overall_death,age_0024_firearm_death,age_0024_nonfire_death,age_0024_overall_smr,age_0024_firearm_smr,age_0024_nonfire_smr
0,1970,Dec,84967089,age 0-24,12,3267,1531,1736,3.845018,1.801874,2.043144
1,1971,Dec,86728328,age 0-24,12,3628,1819,1809,4.183178,2.097354,2.085824
2,1972,Dec,86574634,age 0-24,12,3988,2212,1776,4.606430,2.555021,2.051409
3,1973,Dec,86581713,age 0-24,12,4257,2331,1926,4.916743,2.692254,2.224488
4,1974,Dec,86100981,age 0-24,12,4480,2633,1847,5.203193,3.058037,2.145156


In [87]:
suicide_yearly.head()

,year,pop,month_code,overall_death,firearm_death,nonfirearm_death,overall_smr,firearm_smr,nonfirearm_smr
0,1970,185186282,12,23500,11777,11723,12.689925,6.359542,6.330383
1,1971,188303852,12,24122,12293,11829,12.810147,6.528279,6.281868
2,1972,190864825,12,25028,13352,11676,13.112945,6.995527,6.117418
3,1973,192218457,12,25137,13323,11814,13.077308,6.931176,6.146132
4,1974,194874242,12,25708,14348,11360,13.192097,7.362697,5.829400


In [88]:
age_monthly = age_0024_monthly[['year', 'age_0024_overall_smr', 'age_0024_firearm_smr', 'age_0024_nonfire_smr']].merge(age_2564_monthly[['year', 'age_2564_overall_smr', 'age_2564_firearm_smr', 'age_2564_nonfire_smr']], on = 'year', how = 'inner')
age_monthly = age_monthly.merge(age_65_monthly[['year', 'age_65_overall_smr', 'age_65_firearm_smr', 'age_65_nonfire_smr']], on = 'year', how = 'inner')
age_monthly = age_monthly.merge(suicide_yearly[['year', 'overall_smr', 'firearm_smr', 'nonfirearm_smr']], on = 'year', how = 'inner')
age_monthly.head()

,year,age_0024_overall_smr,age_0024_firearm_smr,age_0024_nonfire_smr,age_2564_overall_smr,age_2564_firearm_smr,age_2564_nonfire_smr,age_65_overall_smr,age_65_firearm_smr,age_65_nonfire_smr,overall_smr,firearm_smr,nonfirearm_smr
0,1970,3.845018,1.801874,2.043144,19.576093,9.859412,9.716681,22.864337,11.821279,11.043058,12.689925,6.359542,6.330383
1,1971,4.183178,2.097354,2.085824,19.538393,9.939400,9.598993,22.958870,11.951851,11.007020,12.810147,6.528279,6.281868
2,1972,4.606430,2.555021,2.051409,19.535907,10.358137,9.177769,22.998020,12.097042,10.900977,13.112945,6.995527,6.117418
3,1973,4.916743,2.692254,2.224488,19.223547,10.076447,9.147100,22.122430,11.833403,10.289027,13.077308,6.931176,6.146132
4,1974,5.203193,3.058037,2.145156,19.182995,10.549519,8.633476,20.949625,11.725240,9.224385,13.192097,7.362697,5.829400


In [89]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = age_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Create a new plot with a title and axis labels
p = figure(title="Yearly Overall Suicide Rates by Age per 100,000 from 1970 to 2020", x_axis_label='year', y_axis_label='Suicide Rates Per 100,000', x_range=(1970, 2020))

# Add a line renderer for each suicide rates
p.line('year', 'age_0024_overall_smr', source=source, line_width=2, line_color="green", legend_label="age 0-24")
p.line('year', 'age_2564_overall_smr', source=source, line_width=2, line_color="red", legend_label="age 25-64")
p.line('year', 'age_65_overall_smr', source=source, line_width=2, line_color="blue", legend_label="age 65+")
p.line('year', 'overall_smr', source=source, line_width=4, line_color="black", legend_label="total average")
# Add HoverTool
hover = HoverTool(
    tooltips=[
        ("year", "@year"),
        ("age_0024_overall_smr", "@{age_0024_overall_smr}"),
        ("age_2564_overall_smr", "@{age_2564_overall_smr}"),
        ("age_65_overall_smr", "@{age_65_overall_smr}"),
        ("overall_smr", "@{overall_smr}")
    ]
)

p.add_tools(hover)
p.y_range.start = 0
p.y_range.end = 30
p.legend.location = "top_right"
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend
# Show the result
show(p)

Loading BokehJS ...

In [90]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = age_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Create a new plot with a title and axis labels
p = figure(title="Yearly Firearm Suicide Rates by Age per 100,000 from 1970 to 2020", x_axis_label='year', y_axis_label='Suicide Rates Per 100,000', x_range=(1970, 2020))

# Add a line renderer for each suicide rates
p.line('year', 'age_0024_firearm_smr', source=source, line_width=2, line_color="green", legend_label="age 0-24")
p.line('year', 'age_2564_firearm_smr', source=source, line_width=2, line_color="red", legend_label="age 25-64")
p.line('year', 'age_65_firearm_smr', source=source, line_width=2, line_color="blue", legend_label="age 65+ ")
p.line('year', 'firearm_smr', source=source, line_width=4, line_color="black", legend_label="total average")
# Add HoverTool
hover = HoverTool(
    tooltips=[
        ("year", "@year"),
        ("age_0024_firearm_smr", "@{age_0024_firearm_smr}"),
        ("age_2564_firearm_smr", "@{age_2564_firearm_smr}"),
        ("age_65_firearm_smr", "@{age_65_firearm_smr}"),
        ("firearm_smr", "@{firearm_smr}")
    ]
)

p.add_tools(hover)
p.y_range.start = 0
p.y_range.end = 30
p.legend.location = "top_right"
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend
p.title.text_font_size = '10pt'
# Show the result
show(p)

Loading BokehJS ...

In [91]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = age_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Create a new plot with a title and axis labels
p = figure(title="Yearly Non-Firearm Suicide Rates by Age per 100,000 from 1970 to 2020", x_axis_label='year', y_axis_label='Suicide Rates Per 100,000', x_range=(1970, 2020))

# Add a line renderer for each suicide rates
p.line('year', 'age_0024_nonfire_smr', source=source, line_width=2, line_color="green", legend_label="age 0-24")
p.line('year', 'age_2564_nonfire_smr', source=source, line_width=2, line_color="red", legend_label="age 25-64")
p.line('year', 'age_65_nonfire_smr', source=source, line_width=2, line_color="blue", legend_label="age 65+ ")
p.line('year', 'nonfirearm_smr', source=source, line_width=4, line_color="black", legend_label="total average")
# Add HoverTool
hover = HoverTool(
    tooltips=[
        ("year", "@year"),
        ("age_0024_nonfire_smr", "@{age_0024_nonfire_smr}"),
        ("age_2564_nonfire_smr", "@{age_2564_nonfire_smr}"),
        ("age_65_nonfire_smr", "@{age_65_nonfire_smr}"),
        ("nonfirearm_smr", "@{nonfirearm_smr}")
    ]
)

p.add_tools(hover)
p.y_range.start = 0
p.y_range.end = 30
p.legend.location = "top_right"
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend
p.title.text_font_size = '10pt'
# Show the result
show(p)

Loading BokehJS ...

In [92]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = age_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['age_0024_overall_smr', 'age_2564_overall_smr', 'age_65_overall_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=(1970, 2020), plot_height=600, title="Average Yearly Overall Suicide Rates in Each Age Group from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='year', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='year', width=0.9, color= ['green', 'orange', 'red'], source=source,
             legend_label=['age 0-24', 'age 25-64', 'age 65+'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 60
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"
p.min_border_top = 10
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend
p.legend.background_fill_alpha = 0.5
# Show the result
show(p)

Loading BokehJS ...

In [93]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = age_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['age_0024_firearm_smr', 'age_2564_firearm_smr', 'age_65_firearm_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=(1970, 2020), plot_height=600, title="Average Yearly Firearm Suicide Rates in Each Age Group from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='year', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='year', width=0.9, color= ['green', 'orange', 'red'], source=source,
             legend_label=['age 0-24', 'age 25-64', 'age 65+'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 60
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"
p.min_border_top = 10
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend
p.legend.background_fill_alpha = 0.5
# Show the result
show(p)

Loading BokehJS ...

In [94]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = age_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['age_0024_nonfire_smr', 'age_2564_nonfire_smr', 'age_65_nonfire_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=(1970, 2020), plot_height=600, title="Average Yearly Non-Firearm Suicide Rates in Each Age Group from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='year', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='year', width=0.9, color= ['green', 'orange', 'red'], source=source,
             legend_label=['age 0-24', 'age 25-64', 'age 65+'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 60
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"
p.min_border_top = 10
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend
p.legend.background_fill_alpha = 0.5
# Show the result
show(p)

Loading BokehJS ...

In [95]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = age_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['age_0024_firearm_smr', 'age_0024_nonfire_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=(1970, 2020), plot_height=600, title="Average Yearly Suicide Rates in Age Group 0-24 from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='year', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='year', width=0.9, color= ['orange', 'green'], source=source,
             legend_label=['firearm', 'non-firearm'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 30
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"
p.min_border_top = 10

# Show the result
show(p)

Loading BokehJS ...

In [96]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = age_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['age_2564_firearm_smr', 'age_2564_nonfire_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=(1970, 2020), plot_height=600, title="Average Yearly Suicide Rates in Age Group 25-64 from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='year', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='year', width=0.9, color= ['orange', 'green'], source=source,
             legend_label=['firearm', 'non-firearm'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 30
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"

# Show the result
show(p)

Loading BokehJS ...

In [97]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = age_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['age_65_firearm_smr', 'age_65_nonfire_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=(1970, 2020), plot_height=600, title="Average Yearly Suicide Rates in Age Group 65+ from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='year', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='year', width=0.9, color= ['orange', 'green'], source=source,
             legend_label=['firearm', 'non-firearm'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 30
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"
# Show the result
show(p)

Loading BokehJS ...

In [98]:
age_0024_overall_month = findMonthlyRate(age_0024_overall, 'age_0024_overall')
age_0024_overall_month.head()

,month,month_code,age_0024_overall_smr
4,Jan,1,0.500777
3,Feb,2,0.450137
7,Mar,3,0.496237
0,Apr,4,0.480264
8,May,5,0.486431


In [99]:
age_2564_overall_month = findMonthlyRate(age_2564_overall, 'age_2564_overall')
age_2564_overall_month.head()

,month,month_code,age_2564_overall_smr
4,Jan,1,1.494730
3,Feb,2,1.372360
7,Mar,3,1.549136
0,Apr,4,1.519636
8,May,5,1.561419


In [100]:
age_65_overall_month = findMonthlyRate(age_65_overall, 'age_65_overall')
age_65_overall_month.head()

,month,month_code,age_65_overall_smr
4,Jan,1,1.559583
3,Feb,2,1.430655
7,Mar,3,1.710724
0,Apr,4,1.714851
8,May,5,1.796218


In [101]:
age_0024_firearm_month = findMonthlyRate(age_0024_firearm, 'age_0024_firearm')
age_0024_firearm_month.head()

,month,month_code,age_0024_firearm_smr
4,Jan,1,0.284994
3,Feb,2,0.252040
7,Mar,3,0.275521
0,Apr,4,0.260563
8,May,5,0.263130


In [102]:
age_2564_firearm_month = findMonthlyRate(age_2564_firearm, 'age_2564_firearm')
age_2564_firearm_month.head()

,month,month_code,age_2564_firearm_smr
4,Jan,1,0.786734
3,Feb,2,0.724035
7,Mar,3,0.799956
0,Apr,4,0.787447
8,May,5,0.795500


In [103]:
age_65_firearm_month = findMonthlyRate(age_65_firearm, 'age_65_firearm')
age_65_firearm_month.head()

,month,month_code,age_65_firearm_smr
4,Jan,1,1.041690
3,Feb,2,0.954338
7,Mar,3,1.123639
0,Apr,4,1.121397
8,May,5,1.168315


In [104]:
age_0024_nonfire_month = findMonthlyRate(age_0024_nonfire, 'age_0024_nonfire')
age_0024_nonfire_month.head()

,month,month_code,age_0024_nonfire_smr
4,Jan,1,0.215783
3,Feb,2,0.198097
7,Mar,3,0.220716
0,Apr,4,0.219702
8,May,5,0.223302


In [105]:
age_2564_nonfire_month = findMonthlyRate(age_2564_nonfire, 'age_2564_nonfire')
age_2564_nonfire_month.head()

,month,month_code,age_2564_nonfire_smr
4,Jan,1,0.707996
3,Feb,2,0.648325
7,Mar,3,0.749180
0,Apr,4,0.732189
8,May,5,0.765919


In [106]:
age_65_nonfire_month = findMonthlyRate(age_65_nonfire, 'age_65_nonfire')
age_65_nonfire_month.head()

,month,month_code,age_65_nonfire_smr
4,Jan,1,0.517893
3,Feb,2,0.476317
7,Mar,3,0.587085
0,Apr,4,0.593454
8,May,5,0.627903


In [107]:
age_0024_monthly = age_0024_overall_month.merge(age_0024_firearm_month, on = ['month', 'month_code'], how = 'inner')
age_0024_monthly = age_0024_monthly.merge(age_0024_nonfire_month, on = ['month', 'month_code'], how = 'inner')
age_2564_monthly = age_2564_overall_month.merge(age_2564_firearm_month, on = ['month', 'month_code'], how = 'inner')
age_2564_monthly = age_2564_monthly.merge(age_2564_nonfire_month, on = ['month', 'month_code'], how = 'inner')
age_65_monthly = age_65_overall_month.merge(age_65_firearm_month, on = ['month', 'month_code'], how = 'inner')
age_65_monthly = age_65_monthly.merge(age_65_nonfire_month, on = ['month', 'month_code'], how = 'inner')
age_monthly = suicide_monthly.merge(age_0024_monthly, on = ['month', 'month_code'], how = 'inner')
age_monthly = age_monthly.merge(age_2564_monthly, on = ['month', 'month_code'], how = 'inner')
age_monthly = age_monthly.merge(age_65_monthly, on = ['month', 'month_code'], how = 'inner')
age_monthly

,month,overall_smr,firearm_smr,nonfirearm_smr,month_code,age_0024_overall_smr,age_0024_firearm_smr,age_0024_nonfire_smr,age_2564_overall_smr,age_2564_firearm_smr,age_2564_nonfire_smr,age_65_overall_smr,age_65_firearm_smr,age_65_nonfire_smr
0,Jan,1.128548,0.630133,0.498415,1,0.500777,0.284994,0.215783,1.494730,0.786734,0.707996,1.559583,1.041690,0.517893
1,Feb,1.031992,0.575394,0.456599,2,0.450137,0.252040,0.198097,1.372360,0.724035,0.648325,1.430655,0.954338,0.476317
2,Mar,1.172092,0.643090,0.529003,3,0.496237,0.275521,0.220716,1.549136,0.799956,0.749180,1.710724,1.123639,0.587085
3,Apr,1.150810,0.630303,0.520508,4,0.480264,0.260563,0.219702,1.519636,0.787447,0.732189,1.714851,1.121397,0.593454
4,May,1.184873,0.641393,0.543480,5,0.486431,0.263130,0.223302,1.561419,0.795500,0.765919,1.796218,1.168315,0.627903
5,Jun,1.144430,0.620220,0.524209,6,0.446104,0.239607,0.206497,1.521755,0.775251,0.746505,1.734525,1.141666,0.592859
6,Jul,1.176744,0.643885,0.532858,7,0.464903,0.250537,0.214365,1.556597,0.801178,0.755419,1.786513,1.191142,0.595371
7,Aug,1.175591,0.637623,0.537968,8,0.473479,0.253551,0.219928,1.555796,0.793660,0.762136,1.761004,1.161534,0.599470
8,Sep,1.124047,0.608247,0.515800,9,0.468568,0.254162,0.214406,1.486522,0.755346,0.731175,1.642334,1.077361,0.564973
9,Oct,1.132013,0.615541,0.516472,10,0.491743,0.266888,0.224855,1.485431,0.761600,0.723831,1.645828,1.074591,0.571237


In [108]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = age_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Create a new plot with a title and axis labels
p = figure(x_range=df['month'], plot_height=400, title="Average Monthly Overall Suicide Rates by Age Groups from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='month', y_axis_label='Suicide Rates per 100,000')

# Add a line renderer for each suicide rates

p.line('month', 'age_0024_overall_smr', source=source, line_width=2, line_color="green", legend_label="age 0-24")
p.line('month', 'age_2564_overall_smr', source=source, line_width=2, line_color="blue", legend_label="age 25-64")
p.line('month', 'age_65_overall_smr', source=source, line_width=2, line_color="red", legend_label="age 65+")
p.line('month', 'overall_smr', source=source, line_width=4, line_color="black", legend_label="total average")

# Set some properties
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.y_range.end = 2.5
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend

# Show the result
show(p)

Loading BokehJS ...

In [109]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = age_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Create a new plot with a title and axis labels
p = figure(x_range=df['month'], plot_height=400, title="Average Monthly Firearm Suicide Rates by Age Groups from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='month', y_axis_label='Suicide Rates per 100,000')

# Add a line renderer for each suicide rates

p.line('month', 'age_0024_firearm_smr', source=source, line_width=2, line_color="green", legend_label="age 0-24")
p.line('month', 'age_2564_firearm_smr', source=source, line_width=2, line_color="blue", legend_label="age 25-64")
p.line('month', 'age_65_firearm_smr', source=source, line_width=2, line_color="red", legend_label="age 65+")
p.line('month', 'firearm_smr', source=source, line_width=4, line_color="black", legend_label="total average")

# Set some properties
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.y_range.end = 2.5
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend
p.legend.background_fill_alpha = 0.5
# Show the result
show(p)

Loading BokehJS ...

In [110]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = age_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Create a new plot with a title and axis labels
p = figure(x_range=df['month'], plot_height=400, title="Average Monthly Non-Firearm Suicide Rates by Age Groups from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='month', y_axis_label='Suicide Rates per 100,000')

# Add a line renderer for each suicide rates

p.line('month', 'age_0024_nonfire_smr', source=source, line_width=2, line_color="green", legend_label="age 0-24")
p.line('month', 'age_2564_nonfire_smr', source=source, line_width=2, line_color="blue", legend_label="age 25-64")
p.line('month', 'age_65_nonfire_smr', source=source, line_width=2, line_color="red", legend_label="age 65+")
p.line('month', 'nonfirearm_smr', source=source, line_width=4, line_color="black", legend_label="total average")

# Set some properties
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.y_range.end = 2.5
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend

# Show the result
show(p)

Loading BokehJS ...

In [111]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = age_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['age_0024_overall_smr', 'age_2564_overall_smr', 'age_65_overall_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=df['month'], plot_height=400, title="Average Monthly Overall Suicide Rates per 100,000 from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='month', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='month', width=0.9, color= ['green', 'orange', 'red'], source=source,
             legend_label=['age 0-24', 'age 25-64', 'age 65+'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 6
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend
# Show the result
show(p)

Loading BokehJS ...

In [112]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = age_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['age_0024_firearm_smr', 'age_2564_firearm_smr', 'age_65_firearm_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=df['month'], plot_height=400, title="Average Monthly Firearm Suicide Rates per 100,000 from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='month', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='month', width=0.9, color= ['green', 'orange', 'red'], source=source,
             legend_label=['age 0-24', 'age 25-64', 'age 65+'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 6
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend
# Show the result
show(p)

Loading BokehJS ...

In [113]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

# Import your dataset and convert to a DataFrame (replace 'your_dataframe' with your actual dataframe)
df = age_monthly

source = ColumnDataSource(df)

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Define the categories
categories = ['age_0024_nonfire_smr', 'age_2564_nonfire_smr', 'age_65_nonfire_smr']

# Create a new plot with a title and axis labels
p = figure(x_range=df['month'], plot_height=400, title="Average Monthly Non-Firearm Suicide Rates per 100,000 from 1970 to 2020",
           toolbar_location=None, tools="hover", tooltips="$name: @$name", x_axis_label='month', y_axis_label='Suicide Rates per 100,000')

# Add a vbar_stack
p.vbar_stack(categories, x='month', width=0.9, color= ['green', 'orange', 'red'], source=source,
             legend_label=['age 0-24', 'age 25-64', 'age 65+'])

# Set some properties
p.y_range.start = 0
p.y_range.end = 6
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"
p.legend.margin = 5  # Add some margin around the legend
p.legend.padding = 5  # Add some padding around the items in the legend
# Show the result
show(p)

Loading BokehJS ...